In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import onnx

In [20]:
#Check if GPU is available and select if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
device = torch.device("cuda")

cuda
True


In [21]:
#definining the model
#proposed function for stress level S = A*sin(V) where S is stress, A is arousal, and V is valence
#since valence represents the positivity of the emotion, it would be inversely proportional to stress as higher valence means a better emotion
#Since higher arousal can be generally translated to 
class StressScanner(torch.nn.Module):
    def __init__(self):
        super(StressScanner, self).__init__()
        self.ConvolutionIn = torch.nn.Conv1d(in_channels=1, out_channels=2,kernel_size=1,padding=0)
        self.ConvHidden1 = torch.nn.Conv1d(in_channels=2, out_channels=4,kernel_size=1,padding=0)
        self.ConvHidden2 = torch.nn.Conv1d(in_channels=4, out_channels=8,kernel_size=1,padding=0)
        self.linear1 = torch.nn.Linear(8, 5)
        self.linear2 = torch.nn.Linear(5, 3)
        self.linearOut = torch.nn.Linear(3, 1)

    def forward(self, x):
        x = F.relu(self.ConvolutionIn(x))
        x = F.relu(self.ConvHidden1(x))
        x = F.relu(self.ConvHidden2(x))
        x = x.squeeze()
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linearOut(x)
        return x

In [22]:
#conversion
torch_model = StressScanner()
model_path = '.\models\StressPredictor-2023-10-08_14-11-15.pth'
onnx_path = '.\models\ONNX-BVP-1.onnx'
torch_model.load_state_dict(torch.load(model_path))
torch_input = torch.randn(1024, 1, 1)
export_output = torch.onnx.export(torch_model, torch_input, onnx_path, verbose=True)


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [23]:
#verify conversion successful
onnx_model = onnx.load(".\models\ONNX-BVP-1.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
#testing the model with onnx runtime
